In [1]:
# !pip install mlflow dagshub catboost optuna

In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split
import mlflow
import dagshub
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor
import optuna

In [3]:
dagshub.init(repo_owner='Aryanupadhyay23', repo_name='Zomato-Food-Delivery-Time-prediction', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=472e3c9e-e9bb-4265-ae57-95b5866a099e&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ff7a4e06da1453b80f363d58e4a513791f7c12848c524e72af8dd878404dad39




Accessing as Aryanupadhyay23

Initialized MLflow to track repo "Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction"

Repository Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction initialized!

In [4]:
# set the tracking uri

mlflow.set_tracking_uri("https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow")

In [5]:
# mlflow experiment

mlflow.set_experiment("CatBoost HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/f3eaed63151543dd8a29c2da40bf9a7b', creation_time=1770611503846, experiment_id='4', last_update_time=1770611503846, lifecycle_stage='active', name='CatBoost HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:
df = pd.read_csv("/content/food_delivery_interim.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38055 entries, 0 to 38054
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rider_age            38055 non-null  float64
 1   rider_ratings        38055 non-null  float64
 2   weather              38055 non-null  object 
 3   traffic_density      38055 non-null  object 
 4   vehicle_condition    38055 non-null  int64  
 5   order_type           38055 non-null  object 
 6   vehicle_type         38055 non-null  object 
 7   multiple_deliveries  38055 non-null  float64
 8   festival             38055 non-null  object 
 9   time_taken           38055 non-null  int64  
 10  city_type            38055 non-null  object 
 11  day_name             38055 non-null  object 
 12  time_of_day          38055 non-null  object 
 13  distance             38055 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 4.1+ MB


In [8]:
df.head()

,rider_age,rider_ratings,weather,traffic_density,vehicle_condition,order_type,vehicle_type,multiple_deliveries,festival,time_taken,city_type,day_name,time_of_day,distance
0,36.0,4.2,fog,jam,2,snack,motorcycle,3.0,no,46,metropolitian,saturday,dinner_peak,10.280582
1,21.0,4.7,stormy,high,1,meal,motorcycle,1.0,no,23,metropolitian,sunday,afternoon,6.242319
2,23.0,4.7,sandstorms,medium,1,drinks,scooter,1.0,no,21,metropolitian,friday,evening_snacks,13.787860
3,34.0,4.3,sandstorms,low,0,buffet,motorcycle,0.0,no,20,metropolitian,sunday,breakfast,2.930258
4,24.0,4.7,fog,jam,1,snack,scooter,1.0,no,41,metropolitian,monday,evening_snacks,19.396618


In [9]:
X = df.drop(columns='time_taken')
y = df['time_taken']

In [10]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
num_cols = ["rider_age","rider_ratings","distance"]

nominal_cat_cols = ["weather","order_type","vehicle_type","festival","city_type","day_name","time_of_day"]

ordinal_cat_cols = ["traffic_density"]

In [12]:
traffic_order = ["low","medium","high","jam"]

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("scale", StandardScaler(), num_cols),
        (
            "nominal_encoder",
            OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False),
            nominal_cat_cols
        ),
        (
            "ordinal_encoder",
            OrdinalEncoder(categories=[traffic_order]),
            ordinal_cat_cols
        )
    ],
    remainder="passthrough",
    n_jobs=-1,
    force_int_remainder_cols=False,
    verbose_feature_names_out=False
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('scale', StandardScaler(),
                                 ['rider_age', 'rider_ratings', 'distance']),
                                ('nominal_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'order_type', 'vehicle_type',
                                  'festival', 'city_type', 'day_name',
                                  'time_of_day']),
                                ('ordinal_encoder',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam']]),
                                 ['traffic_density'])],
                  verbose_feature_names_out=False)

In [14]:
pt = PowerTransformer()

y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

In [15]:
y_train_pt = np.ravel(y_train_pt)
y_test_pt  = np.ravel(y_test_pt)

In [16]:
## pipeline

preprocessing_pipeline = Pipeline(
    steps=[
        ("preprocessor",preprocessor)
    ]
)

preprocessing_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                                   remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['rider_age', 'rider_ratings',
                                                   'distance']),
                                                 ('nominal_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather', 'order_type',
                                                   'vehicle_type', 'festival',
                                                   'city_type', 'day_name',
                                                   'time_of_day']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['low',
                                                                              'medium',
                                                                              'high',
                                                                              'jam']]),
                                                  ['traffic_density'])],
                                   verbose_feature_names_out=False))])

In [17]:
X_train_trans = preprocessing_pipeline.fit_transform(X_train)
X_test_trans = preprocessing_pipeline.transform(X_test)

X_train_trans

,rider_age,rider_ratings,distance,weather_fog,weather_sandstorms,weather_stormy,weather_sunny,weather_windy,order_type_drinks,order_type_meal,...,day_name_tuesday,day_name_wednesday,time_of_day_breakfast,time_of_day_dinner_peak,time_of_day_evening_snacks,time_of_day_late_night,time_of_day_lunch_peak,traffic_density,vehicle_condition,multiple_deliveries
6965,-0.282097,1.164633,-1.211247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,0.0
14052,1.454428,0.849370,0.717574,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,3.0
25717,-0.455749,0.218843,0.144911,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
35085,1.280776,-1.988001,-0.385292,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
5921,0.759818,-0.726947,0.689886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,-1.671317,1.164633,-1.203960,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0
6265,-0.976707,-1.672737,-0.932219,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0
11284,-0.976707,-1.672737,0.653365,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,1.0
860,1.454428,-1.357474,0.757606,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,2.0


In [18]:
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor

In [19]:
def objective(trial):
    with mlflow.start_run(nested=True):

        params = {
            "iterations": trial.suggest_int("iterations", 200, 1500),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 50),
            "random_strength": trial.suggest_float("random_strength", 0, 10),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),

            # GPU settings
            "loss_function": "MAE",
            "task_type": "GPU",
            "devices": "0",

            "random_state": 42,
            "verbose": 0
        }

        # log parameters
        mlflow.log_params(params)

        cat_reg = CatBoostRegressor(**params)

        model = TransformedTargetRegressor(
            regressor=cat_reg,
            transformer=pt
        )

        # fit model
        model.fit(X_train_trans, y_train)

        # predictions
        y_pred_train = model.predict(X_train_trans)
        y_pred_test = model.predict(X_test_trans)

        # cross validation
        cv_score = cross_val_score(
            model,
            X_train_trans,
            y_train,
            cv=5,
            scoring="neg_mean_absolute_error",
            n_jobs=1
        )

        mean_score = -cv_score.mean()

        # log metric
        mlflow.log_metric("cross_val_mae", mean_score)

        return mean_score

In [20]:
# create optuna study
study = optuna.create_study(direction="minimize")

with mlflow.start_run(run_name="best_model"):

    study.optimize(
        objective,
        n_trials=20,
        n_jobs=1,
        show_progress_bar=True
    )

    # log best params and score
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_score", study.best_value)

    # build best CatBoost model
    best_cat = CatBoostRegressor(
        **study.best_params,
        loss_function="MAE",
        task_type="GPU",
        devices="0",
        random_state=42,
        verbose=0
    )

    # wrap with TransformedTargetRegressor
    model = TransformedTargetRegressor(
        regressor=best_cat,
        transformer=pt
    )

    # fit model
    model.fit(X_train_trans, y_train)

    # predictions (already in original scale)
    y_pred_train = model.predict(X_train_trans)
    y_pred_test = model.predict(X_test_trans)

    # cross validation
    scores = cross_val_score(
        model,
        X_train_trans,
        y_train,
        scoring="neg_mean_absolute_error",
        cv=5,
        n_jobs=1
    )

    # log metrics
    mlflow.log_metric(
        "training_mae",
        mean_absolute_error(y_train, y_pred_train)
    )
    mlflow.log_metric(
        "test_mae",
        mean_absolute_error(y_test, y_pred_test)
    )
    mlflow.log_metric(
        "training_r2",
        r2_score(y_train, y_pred_train)
    )
    mlflow.log_metric(
        "test_r2",
        r2_score(y_test, y_pred_test)
    )
    mlflow.log_metric(
        "cross_val_mae",
        -scores.mean()
    )

    # log full pipeline model
    mlflow.sklearn.log_model(
        model,
        artifact_path="model"
    )


[I 2026-02-09 08:23:11,665] A new study created in memory with name: no-name-9dedfb4e-71b7-4cf7-a7b0-5a710b52db3e


  0%|          | 0/20 [00:00<?, ?it/s]

Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


🏃 View run magnificent-koi-924 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4/runs/6306e97dcf8e4c90b4a0d30497e910cc
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4
[I 2026-02-09 08:24:20,619] Trial 0 finished with value: 3.3654206520314967 and parameters: {'iterations': 1056, 'depth': 6, 'learning_rate': 0.01259078470244394, 'l2_leaf_reg': 9.675727555865754, 'random_strength': 7.140764765563218, 'bagging_temperature': 0.10500327582768287}. Best is trial 0 with value: 3.3654206520314967.


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


...

🏃 View run placid-horse-38 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4/runs/afcf7ef97ff14c23ae1e7425edaa2333
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4
[I 2026-02-09 08:55:32,373] Trial 18 finished with value: 3.062635306592334 and parameters: {'iterations': 1335, 'depth': 8, 'learning_rate': 0.13859031932995963, 'l2_leaf_reg': 30.950739418891004, 'random_strength': 1.5464760248761102, 'bagging_temperature': 0.8724350187977243}. Best is trial 10 with value: 3.00532470788069.


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


🏃 View run rogue-kite-398 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4/runs/64ffc14dd5db4f1ab72495d49c52182a
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4
[I 2026-02-09 08:57:12,285] Trial 19 finished with value: 3.0238405883511 and parameters: {'iterations': 894, 'depth': 9, 'learning_rate': 0.06521016899236794, 'l2_leaf_reg': 16.730747544515182, 'random_strength': 7.293640492889695, 'bagging_temperature': 0.5655127014082714}. Best is trial 10 with value: 3.00532470788069.


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
2026/02/09 09:00:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run best_model at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4/runs/2ded84618ca247ff88b17d3c162bc001
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/4


In [21]:
study.best_params

{'iterations': 1466,
 'depth': 10,
 'learning_rate': 0.038080477733221894,
 'l2_leaf_reg': 26.021727151100524,
 'random_strength': 9.561951563676054,
 'bagging_temperature': 0.7315752426106197}

In [22]:
study.best_value

3.00532470788069

In [23]:
# optimization history plot

optuna.visualization.plot_optimization_history(study)

In [24]:
# plot hyperparameter importance plot

optuna.visualization.plot_param_importances(study)

In [25]:
# slice plot

optuna.visualization.plot_slice(study)